In [46]:
import pandas as pd
import geopy.distance as GD

In [47]:
goog = pd.read_csv('/Users/nateoppenheimer/code/willbanny/Location-Analysis/raw_data/features/Wandsworth London Boro.csv')

In [48]:
grid = pd.read_csv('/Users/nateoppenheimer/code/willbanny/Location-Analysis/grid_points_trial_bigQuery.csv')

In [75]:
radius_list = [0, 250, 500, 750]
# radius_list = [750, 500, 250]
grid = grid.rename(columns={"Latitude": "lat", "Longitude": "lng"})
google_engineered = grid.copy()

In [68]:
# -----
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2

def calculate_distance(coord1, coord2):
    # Convert coordinates to radians
    lon1, lat1 = radians(coord1['lng']), radians(coord1['latitude'])
    lon2, lat2 = radians(coord2['lng']), radians(coord2['latitude'])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = 6371 * c  # Radius of the Earth in kilometers

    return distance * 1000  # Convert distance to meters

def count_coordinates_within_distance(coord_df, target_df, distance):
    coord_df_rad = np.radians(coord_df[['lng', 'lat']])
    target_df_rad = np.radians(target_df[['lng', 'lat']])

    lat_diff = target_df_rad['lat'].values[:, np.newaxis] - coord_df_rad['lat'].values
    lon_diff = target_df_rad['lng'].values[:, np.newaxis] - coord_df_rad['lng'].values

    a = np.sin(lat_diff / 2)**2 + np.cos(target_df_rad['lat'].values[:, np.newaxis]) * \
        np.cos(coord_df_rad['lat'].values) * np.sin(lon_diff / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distances = 6371 * c * 1000  # Convert distance to meters

    counts = np.sum((distances >= distance[0]) & (distances <= distance[1]), axis=1)
#     print(counts)
    return counts

In [69]:
finished_df = pd.DataFrame()

In [81]:
features = list(goog['feature_name'].unique())
for i in range(0, len(radius_list)-1):
    for f in features:
        finished_df[f'{f}_{radius_list[i+1]}'] = count_coordinates_within_distance(goog[goog['feature_name']==f][['lat', 'lng']], grid[['lat', 'lng']], radius_list[i:(i+2)])

In [84]:
grid[['lng', 'lat']]

,lng,lat
0,-0.168830,51.483640
1,-0.154381,51.483640
2,-0.161606,51.483640
3,-0.139933,51.483640
4,-0.147157,51.483640
...,...,...
182,-0.140097,51.420653
183,-0.161740,51.420653
184,-0.147312,51.420653
185,-0.140109,51.416154


In [88]:
finished_df

,hospital_500,train_station_500,bus_station_500,park_500,place_of_worship_500,hospital_750,train_station_750,bus_station_750,park_750,place_of_worship_750,hospital_250,train_station_250,bus_station_250,park_250,place_of_worship_250
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
183,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
184,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
185,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [91]:
finished_df.join(grid[['lng', 'lat']])

,hospital_500,train_station_500,bus_station_500,park_500,place_of_worship_500,hospital_750,train_station_750,bus_station_750,park_750,place_of_worship_750,hospital_250,train_station_250,bus_station_250,park_250,place_of_worship_250,lng,lat
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.168830,51.483640
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.154381,51.483640
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.161606,51.483640
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.139933,51.483640
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-0.147157,51.483640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.140097,51.420653
183,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-0.161740,51.420653
184,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.147312,51.420653
185,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.140109,51.416154


In [92]:
finished_df['d_name'] = 'steve'

In [93]:
finished_df

,hospital_500,train_station_500,bus_station_500,park_500,place_of_worship_500,hospital_750,train_station_750,bus_station_750,park_750,place_of_worship_750,hospital_250,train_station_250,bus_station_250,park_250,place_of_worship_250,d_name
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,steve
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,steve
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,steve
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,steve
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,steve
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,steve
183,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,steve
184,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,steve
185,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,steve
